In [ ]:
! mkdir /root/.kaggle
! cp /content/kaggle.json /root/.kaggle
! chmod 600 /root/.kaggle/kaggle.json
! kaggle datasets download -d lava18/google-play-store-apps
! unzip /content/google-play-store-apps.zip

  0% 0.00/1.94M [00:00<?, ?B/s]
100% 1.94M/1.94M [00:00<00:00, 174MB/s]
Archive:  /content/google-play-store-apps.zip
  inflating: googleplaystore.csv     
  inflating: googleplaystore_user_reviews.csv  
  inflating: license.txt             


In [ ]:
! pip3 install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 41 kB/s 
     |████████████████████████████████| 199 kB 66.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=c44e35e86e846b51b6e0bd1f3b12ca79d85e378b304a4e79cb85eb552cf81881
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql.session import SparkSession

spark = SparkSession.builder.getOrCreate()

In [ ]:
apps = spark.read.csv('/content/googleplaystore.csv', inferSchema=True, header=True)
print(apps.count())
apps.show()

10841
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|      Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design|   January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|  January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|    August 1, 2018|             1.2.4|

In [ ]:
reviews = spark.read.csv('/content/googleplaystore_user_reviews.csv', inferSchema=True, header=True)
print(reviews.count())
reviews.show()

64295
+--------------------+--------------------+--------------------+-------------------+----------------------+
|                 App|   Translated_Review|           Sentiment| Sentiment_Polarity|Sentiment_Subjectivity|
+--------------------+--------------------+--------------------+-------------------+----------------------+
|10 Best Foods for...|"I like eat delic...| also ""Best Befo...|           Positive|                   1.0|
|10 Best Foods for...|This help eating ...|            Positive|               0.25|   0.28846153846153844|
|10 Best Foods for...|                 nan|                 nan|                nan|                   nan|
|10 Best Foods for...|Works great espec...|            Positive|                0.4|                 0.875|
|10 Best Foods for...|        Best idea us|            Positive|                1.0|                   0.3|
|10 Best Foods for...|            Best way|            Positive|                1.0|                   0.3|
|10 Best Foods for...|

In [ ]:
apps.createOrReplaceTempView('apps') 

query = "SELECT sum(Reviews) FROM apps"

reviews_count = spark.sql(query)

reviews_count.show()

+-------------+
| sum(Reviews)|
+-------------+
|4.814512636E9|
+-------------+



In [ ]:
reviews.createOrReplaceTempView('reviews') 
apps_group_count = spark.sql("SELECT count(*) as C, App FROM reviews GROUP BY App")
apps_group_count.show()
apps_count = spark.sql("SELECT count(C) FROM (SELECT count(*) as C, App FROM reviews GROUP BY App)")
apps_count.show()

+---+--------------------+
|  C|                 App|
+---+--------------------+
| 40|BaBe+ - Berita In...|
|100|    Basketball Stars|
| 40|Davis's Drug Guid...|
| 40|Find&Save - Local...|
| 40|Bloomberg Profess...|
| 40|   Blue CareOnDemand|
| 40|  Floor Plan Creator|
| 40|Bagan - Myanmar K...|
| 40|     Candy Pop Story|
| 40|  Classic Calculator|
| 40|Cricbuzz - Live C...|
| 40|AOL - News, Mail ...|
| 40|Collage&Add Stick...|
| 40|      Epocrates Plus|
| 40|       File Explorer|
|100|            Arrow.io|
| 40|CALCU™ Stylish Ca...|
| 40|FilterGrid - Cam&...|
| 40|BELONG Beating Ca...|
|260|      Bubble Shooter|
+---+--------------------+
only showing top 20 rows

+--------+
|count(C)|
+--------+
|    1074|
+--------+



In [ ]:
apps.show()

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|      Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design|   January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|  January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|    August 1, 2018|             1.2.4|4.0.3 

In [ ]:
test = spark.sql("SELECT * FROM apps").sample(0.001)
test.show()
test.count()

+--------------------+------------------+------+-------+------------------+----------+----+-----+--------------+----------------+----------------+-----------+------------+
|                 App|          Category|Rating|Reviews|              Size|  Installs|Type|Price|Content Rating|          Genres|    Last Updated|Current Ver| Android Ver|
+--------------------+------------------+------+-------+------------------+----------+----+-----+--------------+----------------+----------------+-----------+------------+
|Titanic App - Fem...|            DATING|   4.5|    120|               10M|      500+|Free|    0|    Mature 17+|          Dating|  August 4, 2018|   1.5-beta|  5.0 and up|
|Abs Training-Burn...|HEALTH_AND_FITNESS|   4.7|   2071|              6.9M|  100,000+|Free|    0|      Everyone|Health & Fitness|   July 24, 2018|     1.0.12|  4.1 and up|
|           HD Camera|       PHOTOGRAPHY|   4.3|  49680|              8.7M|5,000,000+|Free|    0|      Everyone|     Photography|   July 11,

10

# Data Preprocess

## APP

In [ ]:
spark.sql("SELECT App, count(*) AS C FROM apps GROUP BY App ORDER BY C DESC").show()

+--------------------+---+
|                 App|  C|
+--------------------+---+
|              ROBLOX|  9|
|CBS Sports App - ...|  8|
|         8 Ball Pool|  7|
|Duolingo: Learn L...|  7|
|                ESPN|  7|
|    Candy Crush Saga|  7|
|          Helix Jump|  6|
|          Bowmasters|  6|
|        Temple Run 2|  6|
|                Nick|  6|
|      Bubble Shooter|  6|
|      Subway Surfers|  6|
|     Zombie Catchers|  6|
|          slither.io|  6|
|Bleacher Report: ...|  6|
|Sniper 3D Gun Sho...|  6|
|     Viber Messenger|  5|
|          Skyscanner|  5|
|    Farm Heroes Saga|  5|
| Angry Birds Classic|  5|
+--------------------+---+
only showing top 20 rows



In [ ]:
spark.sql("SELECT * FROM apps WHERE App='Nick'").show()

+----+-------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+-----------+-----------+
| App|     Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|    Last Updated|Current Ver|Android Ver|
+----+-------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+-----------+-----------+
|Nick|ENTERTAINMENT|   4.2| 123279| 25M|10,000,000+|Free|    0|  Everyone 10+|Entertainment;Mus...|January 24, 2018|      2.0.8| 4.4 and up|
|Nick|ENTERTAINMENT|   4.2| 123279| 25M|10,000,000+|Free|    0|  Everyone 10+|Entertainment;Mus...|January 24, 2018|      2.0.8| 4.4 and up|
|Nick|ENTERTAINMENT|   4.2| 123279| 25M|10,000,000+|Free|    0|  Everyone 10+|Entertainment;Mus...|January 24, 2018|      2.0.8| 4.4 and up|
|Nick|ENTERTAINMENT|   4.2| 123279| 25M|10,000,000+|Free|    0|  Everyone 10+|Entertainment;Mus...|January 24, 2018|      2.0.8| 4.4 and up|
|Nick|       

## Current Version and Android Ver and Last Updated

In [ ]:
apps = apps.drop('Current Ver', 'Android Ver', 'Last Updated')
apps.show()

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644| 25M|50,000,000+|Free|    0|          Teen|        Art & Design|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|2.8M|   100,000+|Free|    0|      Everyone|Art & Design;Crea...|
|Paper flowers ins...|ART_AND_DESIGN|   4.4|    167|5.6M|    50,

## Category and Genres

In [ ]:
print(spark.sql("SELECT Category FROM apps GROUP BY Category").count())
print(spark.sql("SELECT Genres FROM apps GROUP BY Genres").count())

categorical_group = spark.sql("SELECT Category, count(*) AS C FROM apps GROUP BY Category").toPandas()
categorical_group

36
121


,Category,C
0,EVENTS,64
1,COMICS,60
2,SPORTS,384
3,WEATHER,82
4,VIDEO_PLAYERS,175
5,AUTO_AND_VEHICLES,85
6,PARENTING,60
7,ENTERTAINMENT,149
8,PERSONALIZATION,392
9,HEALTH_AND_FITNESS,340


In [ ]:
apps = apps.where("Category <> '1.9'")
apps = apps.where("Category <> 'Face'")
apps = apps.where("Category <> 'traffic jams'")

apps.groupBy('Category').count().show(36)

+-------------------+-----+
|           Category|count|
+-------------------+-----+
|             EVENTS|   64|
|             COMICS|   60|
|             SPORTS|  384|
|            WEATHER|   82|
|      VIDEO_PLAYERS|  175|
|  AUTO_AND_VEHICLES|   85|
|          PARENTING|   60|
|      ENTERTAINMENT|  149|
|    PERSONALIZATION|  392|
| HEALTH_AND_FITNESS|  340|
|   TRAVEL_AND_LOCAL|  258|
|BOOKS_AND_REFERENCE|  231|
|     FOOD_AND_DRINK|  127|
|        PHOTOGRAPHY|  335|
|           BUSINESS|  460|
|             FAMILY| 1972|
|           SHOPPING|  260|
|     HOUSE_AND_HOME|   88|
|               GAME| 1144|
|          EDUCATION|  156|
|       PRODUCTIVITY|  424|
|              TOOLS|  843|
|          LIFESTYLE|  382|
| NEWS_AND_MAGAZINES|  283|
|            MEDICAL|  463|
|      COMMUNICATION|  387|
|MAPS_AND_NAVIGATION|  136|
|             DATING|  234|
|       traffic jams|    1|
| LIBRARIES_AND_DEMO|   85|
|             BEAUTY|   53|
|             SOCIAL|  295|
|            FINANCE

we should cluster in categories and recommand apps from selected cluster

In [ ]:
for cat in categorical_group['Category']:
  print(cat)
  apps.filter("Category = '"+ cat + "'").groupBy('Genres').count().show()

EVENTS
+------+-----+
|Genres|count|
+------+-----+
|Events|   64|
+------+-----+

COMICS
+-----------------+-----+
|           Genres|count|
+-----------------+-----+
|Comics;Creativity|    1|
|           Comics|   59|
+-----------------+-----+

SPORTS
+------+-----+
|Genres|count|
+------+-----+
|Sports|  384|
+------+-----+

WEATHER
+-------+-----+
| Genres|count|
+-------+-----+
|Weather|   82|
+-------+-----+

VIDEO_PLAYERS
+--------------------+-----+
|              Genres|count|
+--------------------+-----+
|Video Players & E...|    1|
|Video Players & E...|    1|
|Video Players & E...|  173|
+--------------------+-----+

AUTO_AND_VEHICLES
+---------------+-----+
|         Genres|count|
+---------------+-----+
|Auto & Vehicles|   85|
+---------------+-----+

PARENTING
+--------------------+-----+
|              Genres|count|
+--------------------+-----+
|Parenting;Brain G...|    1|
| Parenting;Education|    7|
|Parenting;Music &...|    6|
|           Parenting|   46|
+----------

except Games and Family we dont need Genres column

In [ ]:
apps = apps.drop('Genres')
apps.show()

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644| 25M|50,000,000+|Free|    0|          Teen|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|2.8M|   100,000+|Free|    0|      Everyone|
|Paper flowers ins...|ART_AND_DESIGN|   4.4|    167|5.6M|    50,000+|Free|    0|      Everyone|
|Smoke Effect Phot...|ART_AND_DESIGN|   3.8|    178| 19M|    50,000+|Free|    0|      Everyone|
|    Infinite Painter|ART_AND_DESIGN|   

## Type and price

In [ ]:
spark.sql("SELECT Type, count(*) FROM apps GROUP BY Type").show()

+------+--------+
|  Type|count(1)|
+------+--------+
|     0|       1|
|102248|       1|
|   NaN|       1|
|  Free|   10037|
|  Paid|     800|
|  2509|       1|
+------+--------+



In [ ]:
apps = apps.filter("Type == 'Free' or Type == 'Paid'")

In [ ]:
apps = apps.drop('Type')
apps.show()

+--------------------+--------------+------+-------+----+-----------+-----+--------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Price|Content Rating|
+--------------------+--------------+------+-------+----+-----------+-----+--------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|    0|      Everyone|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|    0|      Everyone|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|    0|      Everyone|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644| 25M|50,000,000+|    0|          Teen|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|2.8M|   100,000+|    0|      Everyone|
|Paper flowers ins...|ART_AND_DESIGN|   4.4|    167|5.6M|    50,000+|    0|      Everyone|
|Smoke Effect Phot...|ART_AND_DESIGN|   3.8|    178| 19M|    50,000+|    0|      Everyone|
|    Infinite Painter|ART_AND_DESIGN|   4.1|  36815| 29M| 1,000,000+|    0|      Everyone|

## Cast Reviews and Price to int

In [ ]:
from pyspark.sql.functions import col

apps = apps.withColumn('Reviews', apps.Reviews.cast("int"))
apps = apps.withColumn('Price', apps.Price.cast("int"))

apps.show()

+--------------------+--------------+------+-------+----+-----------+-----+--------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Price|Content Rating|
+--------------------+--------------+------+-------+----+-----------+-----+--------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|    0|      Everyone|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|    0|      Everyone|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|    0|      Everyone|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644| 25M|50,000,000+|    0|          Teen|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|2.8M|   100,000+|    0|      Everyone|
|Paper flowers ins...|ART_AND_DESIGN|   4.4|    167|5.6M|    50,000+|    0|      Everyone|
|Smoke Effect Phot...|ART_AND_DESIGN|   3.8|    178| 19M|    50,000+|    0|      Everyone|
|    Infinite Painter|ART_AND_DESIGN|   4.1|  36815| 29M| 1,000,000+|    0|      Everyone|

## Cast rating * 10 to int

In [ ]:
from pyspark.sql.functions import udf, lit
from pyspark.sql.types import IntegerType

@udf
def cast_rating(rate):
  return float(rate) * 10


apps = apps.withColumn('Rating', lit(cast_rating(apps.Rating)))

In [ ]:
apps = apps.withColumn('Rating', apps.Rating.cast("int"))
apps.show()

+--------------------+--------------+------+-------+----+-----------+-----+--------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Price|Content Rating|
+--------------------+--------------+------+-------+----+-----------+-----+--------------+
|Photo Editor & Ca...|ART_AND_DESIGN|    41|    159| 19M|    10,000+|    0|      Everyone|
| Coloring book moana|ART_AND_DESIGN|    39|    967| 14M|   500,000+|    0|      Everyone|
|U Launcher Lite –...|ART_AND_DESIGN|    47|  87510|8.7M| 5,000,000+|    0|      Everyone|
|Sketch - Draw & P...|ART_AND_DESIGN|    45| 215644| 25M|50,000,000+|    0|          Teen|
|Pixel Draw - Numb...|ART_AND_DESIGN|    43|    967|2.8M|   100,000+|    0|      Everyone|
|Paper flowers ins...|ART_AND_DESIGN|    44|    167|5.6M|    50,000+|    0|      Everyone|
|Smoke Effect Phot...|ART_AND_DESIGN|    38|    178| 19M|    50,000+|    0|      Everyone|
|    Infinite Painter|ART_AND_DESIGN|    41|  36815| 29M| 1,000,000+|    0|      Everyone|

## Size

In [ ]:
apps = apps.drop('Size')

## Installs

In [ ]:
print(apps.groupBy('Installs').count().count())
apps.groupBy('Installs').count().show()

20
+--------------+-----+
|      Installs|count|
+--------------+-----+
|       50,000+|  479|
|          100+|  719|
|    5,000,000+|  751|
|  100,000,000+|  409|
|1,000,000,000+|   58|
|    1,000,000+| 1578|
|  500,000,000+|   72|
|           10+|  386|
|           50+|  205|
|            5+|   82|
|          500+|  330|
|        1,000+|  907|
|      500,000+|  539|
|            0+|   14|
|        5,000+|  477|
|   10,000,000+| 1252|
|   50,000,000+|  289|
|      100,000+| 1169|
|            1+|   67|
|       10,000+| 1054|
+--------------+-----+



In [ ]:
@udf
def install_handler(installs):
  if installs == '1,000,000,000+' or installs == '500,000,000+' or installs == '100,000,000+':
    return 'very high'
  elif installs == '50,000,000+' or installs == '10,000,000+' or installs == '5,000,000+' or 'installs' == '1,000,000+':
    return 'high'
  elif installs == '500,000+' or installs == '100,000+' or installs == '50,000+' or 'installs' == '10,000+':
    return 'medium'
  else:
    return 'low'

In [ ]:
apps = apps.withColumn('Installs', lit(install_handler(apps.Installs)))
apps.show()

+--------------------+--------------+------+-------+--------+-----+--------------+
|                 App|      Category|Rating|Reviews|Installs|Price|Content Rating|
+--------------------+--------------+------+-------+--------+-----+--------------+
|Photo Editor & Ca...|ART_AND_DESIGN|    41|    159|     low|    0|      Everyone|
| Coloring book moana|ART_AND_DESIGN|    39|    967|  medium|    0|      Everyone|
|U Launcher Lite –...|ART_AND_DESIGN|    47|  87510|    high|    0|      Everyone|
|Sketch - Draw & P...|ART_AND_DESIGN|    45| 215644|    high|    0|          Teen|
|Pixel Draw - Numb...|ART_AND_DESIGN|    43|    967|  medium|    0|      Everyone|
|Paper flowers ins...|ART_AND_DESIGN|    44|    167|  medium|    0|      Everyone|
|Smoke Effect Phot...|ART_AND_DESIGN|    38|    178|  medium|    0|      Everyone|
|    Infinite Painter|ART_AND_DESIGN|    41|  36815|     low|    0|      Everyone|
|Garden Coloring Book|ART_AND_DESIGN|    44|  13791|     low|    0|      Everyone|
|Kid

In [ ]:
apps.groupBy('Installs').count().show()

+---------+-----+
| Installs|count|
+---------+-----+
|      low| 5819|
|     high| 2292|
|   medium| 2187|
|very high|  539|
+---------+-----+



In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

installs_indexer = StringIndexer(inputCol="Installs", outputCol="Installs Index")
 
installs_encoder = OneHotEncoder(inputCol="Installs Index", outputCol="Installs Encoded")


## Content Rating

In [ ]:
apps.groupBy("Content Rating").count().show()

+---------------+-----+
| Content Rating|count|
+---------------+-----+
|        Unrated|    2|
|           Teen| 1208|
|     Mature 17+|  498|
|   Everyone 10+|  413|
|       Everyone| 8713|
|Adults only 18+|    3|
+---------------+-----+



In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

content_rating_indexer = StringIndexer(inputCol="Content Rating", outputCol="Content Rating Index")
 
content_rating_encoder = OneHotEncoder(inputCol="Content Rating Index", outputCol="Content Rating Encoded")


## Vector Assembler

In [ ]:
apps.show()

+--------------------+--------------+------+-------+--------+-----+--------------+
|                 App|      Category|Rating|Reviews|Installs|Price|Content Rating|
+--------------------+--------------+------+-------+--------+-----+--------------+
|Photo Editor & Ca...|ART_AND_DESIGN|    41|    159|     low|    0|      Everyone|
| Coloring book moana|ART_AND_DESIGN|    39|    967|  medium|    0|      Everyone|
|U Launcher Lite –...|ART_AND_DESIGN|    47|  87510|    high|    0|      Everyone|
|Sketch - Draw & P...|ART_AND_DESIGN|    45| 215644|    high|    0|          Teen|
|Pixel Draw - Numb...|ART_AND_DESIGN|    43|    967|  medium|    0|      Everyone|
|Paper flowers ins...|ART_AND_DESIGN|    44|    167|  medium|    0|      Everyone|
|Smoke Effect Phot...|ART_AND_DESIGN|    38|    178|  medium|    0|      Everyone|
|    Infinite Painter|ART_AND_DESIGN|    41|  36815|     low|    0|      Everyone|
|Garden Coloring Book|ART_AND_DESIGN|    44|  13791|     low|    0|      Everyone|
|Kid

In [ ]:
vec_assembler = VectorAssembler(inputCols=["Rating", "Reviews", "Price", "Content Rating Encoded", "Installs Encoded"], outputCol="features")

## Create Pipline

In [ ]:
from pyspark.ml import Pipeline

data_pipeline = Pipeline(stages=[installs_indexer, installs_encoder, content_rating_indexer, content_rating_encoder, vec_assembler])

# Recommender System

In [ ]:
test = test.toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Titanic App - Feminist Dating Application,DATING,4.5,120,10M,500+,Free,0,Mature 17+,Dating,"August 4, 2018",1.5-beta,5.0 and up
1,Abs Training-Burn belly fat,HEALTH_AND_FITNESS,4.7,2071,6.9M,"100,000+",Free,0,Everyone,Health & Fitness,"July 24, 2018",1.0.12,4.1 and up
2,HD Camera,PHOTOGRAPHY,4.3,49680,8.7M,"5,000,000+",Free,0,Everyone,Photography,"July 11, 2018",1.8.5,4.2 and up
3,Flight & Hotel Booking App - ixigo,TRAVEL_AND_LOCAL,4.4,45562,Varies with device,"5,000,000+",Free,0,Everyone,Travel & Local,"July 30, 2018",4.0.9.1,4.1 and up
4,Speech Therapy: F,FAMILY,1.0,1,16M,10+,Paid,$2.99,Everyone,Education,"October 7, 2016",1.0,2.3.3 and up
5,Cp Calculator for Evolution:GO,FAMILY,4.8,124,2.5M,500+,Free,0,Everyone,Entertainment,"August 23, 2016",1.2,4.2 and up
6,Radiology CT Anatomy,MEDICAL,4.2,345,14M,"50,000+",Free,0,Everyone,Medical,"January 20, 2018",1.6,4.0 and up
7,D.C. Driving/Walking Tours,TRAVEL_AND_LOCAL,NaN,0,32M,50+,Paid,$4.99,Everyone,Travel & Local,"July 27, 2018",3.8.0,4.4 and up
8,Wi-Fi Master,TOOLS,4.5,3673,5.8M,"500,000+",Free,0,Everyone,Tools,"July 24, 2018",1.2.1,4.1 and up
9,HD Themes Toyota Cruiser 80,PERSONALIZATION,NaN,2,13M,100+,Free,0,Teen,Personalization,"October 3, 2016",1.0,2.3.3 and up


In [ ]:
def recommender_system(app):
  data = apps.filter("Category == '" + str(app.Category) + "'")
  data = data.na.fill(value=0)

  piped_data = data_pipeline.fit(data).transform(data)

  from pyspark.ml.clustering import KMeans
  from pyspark.ml.evaluation import ClusteringEvaluator
  
  evaluator = ClusteringEvaluator(predictionCol='prediction', featuresCol='features', \
                                  metricName='silhouette', distanceMeasure='squaredEuclidean')
  for i in range(2,10):
      try:
        KMeans_algo=KMeans(featuresCol='features', k=i)
        
        KMeans_fit=KMeans_algo.fit(piped_data)
        
        output=KMeans_fit.transform(piped_data)
      except:
        continue
      
      
      score=evaluator.evaluate(output)
            
      print("Silhouette Score:",score)

  return output

clustred = recommender_system(test.iloc[2])
clustred.show()

Silhouette Score: 0.9551464857816384
Silhouette Score: 0.9240468653522671
Silhouette Score: 0.9072921828120443
Silhouette Score: 0.912388353100802
Silhouette Score: 0.8816183592413354
+--------------------+-----------+------+--------+---------+-----+--------------+--------------+----------------+--------------------+----------------------+--------------------+----------+
|                 App|   Category|Rating| Reviews| Installs|Price|Content Rating|Installs Index|Installs Encoded|Content Rating Index|Content Rating Encoded|            features|prediction|
+--------------------+-----------+------+--------+---------+-----+--------------+--------------+----------------+--------------------+----------------------+--------------------+----------+
|TouchNote: Cards ...|PHOTOGRAPHY|    41|   19232|      low|    0|      Everyone|           1.0|   (3,[1],[1.0])|                 0.0|         (2,[0],[1.0])|(8,[0,1,3,6],[41....|         0|
|Shutterfly: Free ...|PHOTOGRAPHY|    46|   98716|     h

In [ ]:
clustred.filter("App == '" + test.iloc[2].App +"'").show()

+---------+-----------+------+-------+--------+-----+--------------+--------------+----------------+--------------------+----------------------+--------------------+----------+
|      App|   Category|Rating|Reviews|Installs|Price|Content Rating|Installs Index|Installs Encoded|Content Rating Index|Content Rating Encoded|            features|prediction|
+---------+-----------+------+-------+--------+-----+--------------+--------------+----------------+--------------------+----------------------+--------------------+----------+
|HD Camera|PHOTOGRAPHY|    43|  49680|    high|    0|      Everyone|           0.0|   (3,[0],[1.0])|                 0.0|         (2,[0],[1.0])|(8,[0,1,3,5],[43....|         0|
+---------+-----------+------+-------+--------+-----+--------------+--------------+----------------+--------------------+----------------------+--------------------+----------+



prediction is cluster 0

In [ ]:
print("clusters:")
clustred.groupBy("prediction").count().show()
print("Recommendations:")
clustred.filter("prediction == 0").sample(0.1).show()

clusters:
+----------+-----+
|prediction|count|
+----------+-----+
|         1|    4|
|         6|   15|
|         3|    5|
|         5|   28|
|         4|    3|
|         8|    7|
|         7|   37|
|         2|   10|
|         0|  226|
+----------+-----+

Recommendations:
+--------------------+-----------+------+-------+--------+-----+--------------+--------------+----------------+--------------------+----------------------+--------------------+----------+
|                 App|   Category|Rating|Reviews|Installs|Price|Content Rating|Installs Index|Installs Encoded|Content Rating Index|Content Rating Encoded|            features|prediction|
+--------------------+-----------+------+-------+--------+-----+--------------+--------------+----------------+--------------------+----------------------+--------------------+----------+
|Moony Lab - Print...|PHOTOGRAPHY|    47|   1320|  medium|    0|      Everyone|           2.0|   (3,[2],[1.0])|                 0.0|         (2,[0],[1.0])|(8,[0,

In [ ]:
spark.sql("SELECT * FROM apps WHERE App='Telegram'").show()

+--------+-------------+------+-------+------------------+------------+----+-----+--------------+-------------+-------------+------------------+------------------+
|     App|     Category|Rating|Reviews|              Size|    Installs|Type|Price|Content Rating|       Genres| Last Updated|       Current Ver|       Android Ver|
+--------+-------------+------+-------+------------------+------------+----+-----+--------------+-------------+-------------+------------------+------------------+
|Telegram|COMMUNICATION|   4.4|3128250|Varies with device|100,000,000+|Free|    0|    Mature 17+|Communication|July 27, 2018|Varies with device|Varies with device|
|Telegram|COMMUNICATION|   4.4|3128509|Varies with device|100,000,000+|Free|    0|    Mature 17+|Communication|July 27, 2018|Varies with device|Varies with device|
|Telegram|COMMUNICATION|   4.4|3128611|Varies with device|100,000,000+|Free|    0|    Mature 17+|Communication|July 27, 2018|Varies with device|Varies with device|
+--------+------

In [ ]:
spark.sql("SELECT * FROM apps WHERE Category='COMMUNICATION' AND Installs='100,000,000+' AND Rating > 4.3").show()

+--------------------+-------------+------+-------+------------------+------------+----+-----+--------------+-------------+--------------+------------------+------------------+
|                 App|     Category|Rating|Reviews|              Size|    Installs|Type|Price|Content Rating|       Genres|  Last Updated|       Current Ver|       Android Ver|
+--------------------+-------------+------+-------+------------------+------------+----+-----+--------------+-------------+--------------+------------------+------------------+
|Messenger Lite: F...|COMMUNICATION|   4.4|1429035|Varies with device|100,000,000+|Free|    0|      Everyone|Communication| July 25, 2018|      37.0.0.7.163|        2.3 and up|
|Firefox Browser f...|COMMUNICATION|   4.4|3075028|Varies with device|100,000,000+|Free|    0|      Everyone|Communication| July 10, 2018|Varies with device|Varies with device|
|Opera Mini - fast...|COMMUNICATION|   4.5|5149854|Varies with device|100,000,000+|Free|    0|      Everyone|Commun